In [1]:
# READ DATA
import pandas as pd
dfs = pd.read_excel('OnlineRetail.xlsx', sheet_name='OnlineRetail')

In [2]:
# Checkout the data format
dfs.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [91]:
# statistics of Data
print dfs.describe()

            Quantity      UnitPrice     CustomerID    AmountSpend
count  397924.000000  397924.000000  397924.000000  397924.000000
mean       13.021823       3.116174   15294.315171      22.394749
std       180.420210      22.096788    1713.169877     309.055588
min         1.000000       0.000000   12346.000000       0.000000
25%         2.000000       1.250000   13969.000000       4.680000
50%         6.000000       1.950000   15159.000000      11.800000
75%        12.000000       3.750000   16795.000000      19.800000
max     80995.000000    8142.750000   18287.000000  168469.600000


In [4]:
# The first thing to notice is that we have some negative values in our dataset.
# Maybe, this could mean that these quantity was returned or cancelled. (Same for UnitPrice negative values)

In [5]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.8+ MB


In [6]:
#Our dataset is composed of 541909 rows and 8 columns. 
#When going through the columns, we notice that we have some missing values in the CustomerID column, 
#as it only has 406829 values. Also the column Description has missing values. 
#So, that should be considered when applying exploration to our dataset.

In [7]:
#UNIQUE Values 
#exploring the unique values of each attribute
print("Number of transactions: ", dfs['InvoiceNo'].nunique())
print("Number of products bought: ",dfs['StockCode'].nunique())
print("Number of customers:", dfs['CustomerID'].nunique() )
print("Percentage of customers NA: ", round(dfs['CustomerID'].isnull().sum() * 100 / len(dfs),2),"%" )
print('Number of countries: ',dfs['Country'].nunique())



('Number of transactions: ', 25900)
('Number of products bought: ', 4070)
('Number of customers:', 4372)
('Percentage of customers NA: ', 24.0, '%')
('Number of countries: ', 38)


In [8]:


# Note: The number of NA customers is quite large and that would impact the results.

# This dataframe contains 8 variables that correspond to:

# InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
# StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
# Description: Product (item) name. Nominal.
# Quantity: The quantities of each product (item) per transaction. Numeric.
# InvoiceDate: Invoice Date and time. Numeric, the day and time when each transaction was generated.
# UnitPrice: Unit price. Numeric, Product price per unit in sterling.
# CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
# Country: Country name. Nominal, the name of the country where each customer resides.

#Now, let's have an idea about the quantitative data (Quantity & UnitPrice).


In [9]:
#df.isnull().any(), you can find just the columns that have NaN values:
dfs.isnull().any()

InvoiceNo      False
StockCode      False
Description     True
Quantity       False
InvoiceDate    False
UnitPrice      False
CustomerID      True
Country        False
dtype: bool

In [10]:
#df.isnull().sum().sum() - This returns an integer of the total number of NaN values
dfs.isnull().sum().sum()

136534

In [11]:
##df.isnull().sum() - This returns an integer of the number of NaN values per Column
dfs.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [12]:
# Data Pre-Processing Steps
######################------------------------------#####################


In [13]:
# Filter out all the irrelevant columns from the Dataframe
# axis=1 denotes that we are referring to a column, not a row
# Dropping columns InvoiceNo, Description, Country, InvoiceDate

In [14]:
#dfs.drop('InvoiceNo', axis = 1)
dfs.index

RangeIndex(start=0, stop=541909, step=1)

In [15]:
Datframe_dropcolms  = dfs.drop(['Description', 'Country'], axis=1)

In [16]:
Datframe_dropcolms.dtypes

InvoiceNo              object
StockCode              object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
dtype: object

In [17]:
#######--------------------------------------------------------------------------------------------------------------#####
# Cancelled invoices

# As mentioned in the description of the dataset, some InvoiceNo start with the letter "c" = cancelled. Let's see if our hypothesis is correct about the negative quantity: -80995.
# We will look for the list of cancelled invoices and check if there is an invoice with that quantity.


In [18]:
#get canceled transactions
cancelled_orders = dfs[dfs['InvoiceNo'].astype(str).str.contains('C')]
cancelled_orders.head()



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


In [19]:
# How much canceled orders do we have?

In [20]:
#check how many rows our dataframe of cancelled orders contain
print("We have ",len(cancelled_orders), " cancelled orders.")
#percentage of cancelled orders in total orders
total_orders = dfs['InvoiceNo'].nunique()
cancelled_number = len(cancelled_orders)
print('Percentage of orders canceled: {}/{} ({:.2f}%) '.format(cancelled_number, total_orders, cancelled_number/total_orders*100))

('We have ', 9288, ' cancelled orders.')
Percentage of orders canceled: 9288/25900 (0.00%) 


In [21]:
#remove canceled orders
dfs = dfs[dfs['Quantity']>0]
dfs.shape

(531285, 8)

In [22]:
#remove rows where customerID are NA
dfs.dropna(subset=['CustomerID'],how='all',inplace=True)
dfs.shape

(397924, 8)

In [23]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      397924 non-null object
StockCode      397924 non-null object
Description    397924 non-null object
Quantity       397924 non-null int64
InvoiceDate    397924 non-null datetime64[ns]
UnitPrice      397924 non-null float64
CustomerID     397924 non-null float64
Country        397924 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 21.3+ MB


In [24]:
##### Build the Matrix [User,Items]
List_Unique_items = dfs['Description'].unique()
List_Unique_items = dfs['StockCode'].unique()

In [25]:
List_Unique_items.tolist()

[u'85123A',
 71053,
 u'84406B',
 u'84029G',
 u'84029E',
 22752,
 21730,
 22633,
 22632,
 84879,
 22745,
 22748,
 22749,
 22310,
 84969,
 22623,
 22622,
 21754,
 21755,
 21777,
 48187,
 22960,
 22913,
 22912,
 22914,
 21756,
 22728,
 22727,
 22726,
 21724,
 21883,
 10002,
 21791,
 21035,
 22326,
 22629,
 22659,
 22631,
 22661,
 21731,
 22900,
 21913,
 22540,
 22544,
 22492,
 u'POST',
 22086,
 20679,
 37370,
 21871,
 21071,
 21068,
 82483,
 82486,
 82482,
 u'82494L',
 21258,
 22114,
 21733,
 22386,
 u'85099C',
 21033,
 20723,
 u'84997B',
 u'84997C',
 21094,
 20725,
 21559,
 22352,
 21212,
 21975,
 21977,
 84991,
 u'84519A',
 u'85183B',
 u'85071B',
 21931,
 21929,
 22961,
 22139,
 84854,
 22411,
 82567,
 21672,
 22774,
 22771,
 71270,
 22262,
 22637,
 21934,
 21169,
 21166,
 21175,
 u'37444A',
 u'37444C',
 22083,
 u'84971S',
 47580,
 22261,
 84832,
 22644,
 21533,
 21557,
 u'15056BL',
 u'15056N',
 22646,
 22176,
 22438,
 22778,
 22719,
 21523,
 21912,
 21832,
 22379,
 22381,
 22798,
 2292

In [26]:
List_Unique_items.shape

(3665,)

In [27]:
List_Unique_users = dfs['CustomerID'].unique()
List_Unique_users.shape

(4339,)

In [28]:
###### Multiplying Quantity and UnitPrice columns to get a new column : AmountSpend########
dfs['AmountSpend'] = dfs['Quantity']*dfs['UnitPrice']

In [29]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 9 columns):
InvoiceNo      397924 non-null object
StockCode      397924 non-null object
Description    397924 non-null object
Quantity       397924 non-null int64
InvoiceDate    397924 non-null datetime64[ns]
UnitPrice      397924 non-null float64
CustomerID     397924 non-null float64
Country        397924 non-null object
AmountSpend    397924 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 24.3+ MB


In [30]:
dfs.tail(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,AmountSpend
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,14.85


In [31]:
#### Check if there is a negative AmountSpend in the filtered database
#  len(DataFrame.index) is equal to number of rows
negative_amountSpend = dfs[dfs['AmountSpend'] < 0]
len(negative_amountSpend.index)

0

In [32]:
###  Here we can see that there is no negative amount spend
## now to think about how to create User, Item matrix 
## My approach is to consider how much money did each User spent on each Item
## 


In [33]:
## Keeping relevant coluns from the Database
matrix_setup = dfs[['StockCode','CustomerID','AmountSpend']]
matrix_setup.shape

(397924, 3)

In [34]:
matrix_setup.head(10)

,StockCode,CustomerID,AmountSpend
0,85123A,17850.0,15.30
1,71053,17850.0,20.34
2,84406B,17850.0,22.00
3,84029G,17850.0,20.34
4,84029E,17850.0,20.34
5,22752,17850.0,15.30
6,21730,17850.0,25.50
7,22633,17850.0,11.10
8,22632,17850.0,11.10
9,84879,13047.0,54.08


In [35]:
dfs.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            541899, 541900, 541901, 541902, 541903, 541904, 541905, 541906,
            541907, 541908],
           dtype='int64', length=397924)

In [38]:
List_Customers = dfs['CustomerID'].unique()
List_Items = dfs['StockCode'].unique()

In [41]:
matrix_setup.shape

(397924, 3)

In [42]:
# Numpy matrix  uses 'iloc' to identify row and column, location of the cell
#.iloc is primarily integer position based (from 0 to length-1 of the axis)
print(matrix_setup.iloc[0][0])
print(matrix_setup.iloc[0][1])
print(matrix_setup.iloc[0][2])

85123A
17850.0
15.299999999999999


In [44]:
matrix_setup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 3 columns):
StockCode      397924 non-null object
CustomerID     397924 non-null float64
AmountSpend    397924 non-null float64
dtypes: float64(2), object(1)
memory usage: 10.6+ MB


In [61]:
# implementing SVD and SVD++ with scikit-surprise
from surprise import SVD,SVDpp,evaluate

In [93]:
#The name SurPRISE (roughly :) ) stands for Simple Python RecommendatIon System Engine.
from surprise import SVD
from surprise.dataset import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.dataset import DatasetAutoFolds

In [84]:
# DatasetAutoFolds.split : 	Split the dataset into folds for future cross-validation.
#DatasetAutoFolds(ratings_file=None, reader=None, df=None)
# A derived class from Dataset for which folds (for cross-validation) are not predefined.
# (Or for when there are no folds at all).
#data = DatasetAutoFolds(ratings_file=None, reader=None, df=df)

In [110]:
#classmethod load_from_df(df, reader)
#reader = Reader(rating_scale=(0, 1))
#datac = Dataset.load_from_df(df=df,reader = None)

In [112]:
#def load_from_df(cls, df, reader):
#        """Load a dataset from a pandas dataframe.
 #       Use this if you want to use a custom dataset that is stored in a pandas
#        dataframe. See the :ref:`User Guide<load_from_df_example>` for an
 #       example.
 #       Args:
 #           df(`Dataframe`): The dataframe containing the ratings. It must have
 #               three columns, corresponding to the user (raw) ids, the item
  #              (raw) ids, and the ratings, in this order.
  #          reader(:obj:`Reader <surprise.reader.Reader>`): A reader to read
  #              the file. Only the ``rating_scale`` field needs to be
  #              specified.

In [142]:
# now can we use matrix_setup datafrmae for Surprise Libraryand use SVD,SVD++, NNMF

#define the reader
reader = Reader(rating_scale= (0,1))

# Assigning data as matrix_setup
data = Dataset.load_from_df(df=matrix_setup[['CustomerID','StockCode','AmountSpend']],reader=reader)



In [143]:
# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     23.9929 20.9757 21.1114 20.4354 20.5725 21.4176 1.3115  
RMSE (testset)    662.2427155.689579.6912 69.4760 76.6870 208.7573228.8952
Fit time          901.59  900.94  901.74  900.15  899.38  900.76  0.89    
Test time         1.03    0.60    0.60    0.59    0.58    0.68    0.17    


{u'fit_time': (901.5880000591278,
  900.9370000362396,
  901.7389998435974,
  900.1480000019073,
  899.375),
 u'test_mae': array([23.9928946 , 20.97572935, 21.1114436 , 20.43543909, 20.5724613 ]),
 u'test_rmse': array([662.24270931, 155.68951953,  79.69123537,  69.47598591,
         76.68695806]),
 u'test_time': (1.0310001373291016,
  0.6040000915527344,
  0.6019999980926514,
  0.5940001010894775,
  0.5780000686645508)}

In [127]:
# swap columns so that userID is the first column
matrix_toPass = matrix_setup[['CustomerID','StockCode','AmountSpend']]

In [133]:
matrix_toPass.head()

,CustomerID,StockCode,AmountSpend
0,17850.0,85123A,15.30
1,17850.0,71053,20.34
2,17850.0,84406B,22.00
3,17850.0,84029G,20.34
4,17850.0,84029E,20.34


In [134]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import surprise; print("surprise", surprise.__version__)

Windows-7-6.1.7601-SP1
('Python', '2.7.14 |Anaconda, Inc.| (default, Nov  8 2017, 13:40:13) [MSC v.1500 32 bit (Intel)]')
('surprise', '1.0.5')


<surprise.reader.Reader instance at 0x1403D698>

In [137]:
matrix_toPass['AmountSpend'].describe()

count    397924.000000
mean         22.394749
std         309.055588
min           0.000000
25%           4.680000
50%          11.800000
75%          19.800000
max      168469.600000
Name: AmountSpend, dtype: float64